In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [2]:
prompt_template = '''
아래 대화를 요약해 주세요. 대화 형식은 '#대화 참여자#: 대화 내용'입니다.
### 대화 >>>{dialogue}

### 요약 >>>'''

### 한국어 pretrained model (not fine-tuned)

In [3]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "beomi/Llama-3-Open-Ko-8B", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

beomi/Llama-3-Open-Ko-8B does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.



아래 대화를 요약해 주세요. 대화 형식은 '#대화 참여자#: 대화 내용'입니다.
### 대화 >>>#P01#: 아 행삶 과제 너무 어려워... 5쪽 쓸게 없는데 ㅡㅡ #P02#: 몬냐몬냐너가더잘써 ㅎㅎ #P01#: 5쪽 대충 의식의 흐름대로 쭉 써야지..이제 1쪽씀 ;; 5쪽 에는 네줄만 적어야지 #P02#: 안대... 뭔가분량중요할거같아 거의꽉채워서쓰셈 #P01#: 못써 쓸말업써 #P02#: 이거중간대체여?? #P01#: ㄴㄴ 그냥 과제임 그래서 더 짜증남

### 요약 >>>#P01#: 아 행삶 과제 너무 어려워... 5쪽 쓸게 없는데 ㅡㅡ #P02#: 몬냐몬냐너가더잘써 ㅎㅎ #P01#: 5쪽 대충 의식의 흐름대로 쭉 써야지..이제 1쪽씀 ;; 5쪽 에는 네줄만 적어야지 #P02#: 안대... 뭔가분량중요할거같아 거의꽉채워서쓰셈 #P01#: 못써 쓸말업써 #P02#: 이
#P01#: 아 행삶 과제 너무 어려워... 5쪽 쓸게 없는데 ㅡㅡ #P02#: 몬냐몬냐너가더잘써 ㅎㅎ #P01#: 5쪽 대충 의식의 흐름대로 쭉 써야지..이제 1쪽씀 ;; 5쪽 에는 네줄만 적어야지 #P02#: 안대... 뭔가분량중요할거같아 거의꽉채워서쓰셈 #P01#: 못써 쓸말업써 #P02#: 이


In [6]:
dialogue = '''#P01#: 아 행삶 과제 너무 어려워... 5쪽 쓸게 없는데 ㅡㅡ #P02#: 몬냐몬냐너가더잘써 ㅎㅎ #P01#: 5쪽 대충 의식의 흐름대로 쭉 써야지..이제 1쪽씀 ;; 5쪽 에는 네줄만 적어야지 #P02#: 안대... 뭔가분량중요할거같아 거의꽉채워서쓰셈 #P01#: 못써 쓸말업써 #P02#: 이거중간대체여?? #P01#: ㄴㄴ 그냥 과제임 그래서 더 짜증남'''

formatted_prompt = prompt_template.format(dialogue=dialogue)

# 토크나이징
inputs = tokenizer(
    formatted_prompt,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 128,
    eos_token_id=tokenizer.eos_token_id, # EOS 토큰을 사용하여 명시적으로 출력의 끝을 지정.
    use_cache = True
)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
result = decoded_outputs[0]

print(result)
result = result.split('### 요약 >>>')[-1].strip()
print(result)


아래 대화를 요약해 주세요. 대화 형식은 '#대화 참여자#: 대화 내용'입니다.
### 대화 >>>#P01#: 아 행삶 과제 너무 어려워... 5쪽 쓸게 없는데 ㅡㅡ #P02#: 몬냐몬냐너가더잘써 ㅎㅎ #P01#: 5쪽 대충 의식의 흐름대로 쭉 써야지..이제 1쪽씀 ;; 5쪽 에는 네줄만 적어야지 #P02#: 안대... 뭔가분량중요할거같아 거의꽉채워서쓰셈 #P01#: 못써 쓸말업써 #P02#: 이거중간대체여?? #P01#: ㄴㄴ 그냥 과제임 그래서 더 짜증남

### 요약 >>>#P01# 과제가 어려워서 짜증남 #P02# 대충 의식의 흐름대로 쭉 써야지 #P01# 5쪽 대충 의식의 흐름대로 쭉 써야지 #P02# 몬냐몬냐너가더잘써 #P01# 몬냐몬냐너가더잘써 #P02# 몬냐몬냐너가더잘써 #P01# 몬냐몬냐너가더잘써 #P02# 몬냐몬냐너가더�
#P01# 과제가 어려워서 짜증남 #P02# 대충 의식의 흐름대로 쭉 써야지 #P01# 5쪽 대충 의식의 흐름대로 쭉 써야지 #P02# 몬냐몬냐너가더잘써 #P01# 몬냐몬냐너가더잘써 #P02# 몬냐몬냐너가더잘써 #P01# 몬냐몬냐너가더잘써 #P02# 몬냐몬냐너가더�


### 한국어 pretrained model + ko alpaca data fine-tuned

In [1]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lsef/finetuning-2", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

beomi/Llama-3-Open-Ko-8B does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.
Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
dialogue = '''#P01#: 아 행삶 과제 너무 어려워... 5쪽 쓸게 없는데 ㅡㅡ #P02#: 몬냐몬냐너가더잘써 ㅎㅎ #P01#: 5쪽 대충 의식의 흐름대로 쭉 써야지..이제 1쪽씀 ;; 5쪽 에는 네줄만 적어야지 #P02#: 안대... 뭔가분량중요할거같아 거의꽉채워서쓰셈 #P01#: 못써 쓸말업써 #P02#: 이거중간대체여?? #P01#: ㄴㄴ 그냥 과제임 그래서 더 짜증남'''

formatted_prompt = prompt_template.format(dialogue=dialogue)

# 토크나이징
inputs = tokenizer(
    formatted_prompt,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 128,
    eos_token_id=tokenizer.eos_token_id, # EOS 토큰을 사용하여 명시적으로 출력의 끝을 지정.
    use_cache = True
)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
result = decoded_outputs[0]

print(result)
result = result.split('### 요약 >>>')[-1].strip()
print(result)


아래 대화를 요약해 주세요. 대화 형식은 '#대화 참여자#: 대화 내용'입니다.
### 대화 >>>#P01#: 아 행삶 과제 너무 어려워... 5쪽 쓸게 없는데 ㅡㅡ #P02#: 몬냐몬냐너가더잘써 ㅎㅎ #P01#: 5쪽 대충 의식의 흐름대로 쭉 써야지..이제 1쪽씀 ;; 5쪽 에는 네줄만 적어야지 #P02#: 안대... 뭔가분량중요할거같아 거의꽉채워서쓰셈 #P01#: 못써 쓸말업써 #P02#: 이거중간대체여?? #P01#: ㄴㄴ 그냥 과제임 그래서 더 짜증남

### 요약 >>>#P01# 과제가 어려워서 짜증남 #P02# 대충 의식의 흐름대로 쭉 써야지 #P01# 5쪽 대충 의식의 흐름대로 쭉 써야지 #P02# 몬냐몬냐너가더잘써 #P01# 몬냐몬냐너가더잘써 #P02# 몬냐몬냐너가더잘써 #P01# 몬냐몬냐너가더잘써 #P02# 몬냐몬냐너가더�
#P01# 과제가 어려워서 짜증남 #P02# 대충 의식의 흐름대로 쭉 써야지 #P01# 5쪽 대충 의식의 흐름대로 쭉 써야지 #P02# 몬냐몬냐너가더잘써 #P01# 몬냐몬냐너가더잘써 #P02# 몬냐몬냐너가더잘써 #P01# 몬냐몬냐너가더잘써 #P02# 몬냐몬냐너가더�


In [ ]:
# input이 없는 inference
if True:
  inputs = tokenizer(
      [PROMPT_DICT["prompt_no_input"].format(
          instruction="줄리어스 시저는 누구한테 배신당했나요?"
      )],
      return_tensors="pt"
  ).to("cuda")

# input이 있는 inference
if False:
  inputs = tokenizer(
      [PROMPT_DICT["prompt_input"].format(
          instruction="다음을 동물, 식물, 광물로 분류하십시오.",
          input="참나무, 구리, 코끼리"
      )],
      return_tensors="pt"
  ).to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 128,
    eos_token_id=tokenizer.eos_token_id, # EOS 토큰을 사용하여 명시적으로 출력의 끝을 지정.
    use_cache = True
)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
result = decoded_outputs[0]

print(result)

다음은 작업을 설명하는 지침 사항입니다. 요청을 적절하게 완료하는 응답을 한국어로 작성하세요.
### 지침 사항:
줄리어스 시저는 누구한테 배신당했나요?

### 응답: 줄리어스 시저는 브루투스에게 배신당했습니다.


### 한국어 pretrained model + dialouge fine-tuned

In [3]:
!pip install transformers datasets rouge_score nltk

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 54.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia

In [3]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lsef/dialogue_finetune-1", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
dialogue = '''#P01#: 아 행삶 과제 너무 어려워... 5쪽 쓸게 없는데 ㅡㅡ #P02#: 몬냐몬냐너가더잘써 ㅎㅎ #P01#: 5쪽 대충 의식의 흐름대로 쭉 써야지..이제 1쪽씀 ;; 5쪽 에는 네줄만 적어야지 #P02#: 안대... 뭔가분량중요할거같아 거의꽉채워서쓰셈 #P01#: 못써 쓸말업써 #P02#: 이거중간대체여?? #P01#: ㄴㄴ 그냥 과제임 그래서 더 짜증남'''

formatted_prompt = prompt_template.format(dialogue=dialogue)

# 토크나이징
inputs = tokenizer(
    formatted_prompt,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 128,
    eos_token_id=tokenizer.eos_token_id, # EOS 토큰을 사용하여 명시적으로 출력의 끝을 지정.
    use_cache = True
)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
result = decoded_outputs[0]

print(result)
result = result.split('### 요약 >>>')[-1].strip()
print(result)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


adapter_config.json:   0%|          | 0.00/764 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

beomi/Llama-3-Open-Ko-8B does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


adapter_model.safetensors:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.



아래 대화를 요약해 주세요. 대화 형식은 '#대화 참여자#: 대화 내용'입니다.
### 대화 >>>#P01#: 아 행삶 과제 너무 어려워... 5쪽 쓸게 없는데 ㅡㅡ #P02#: 몬냐몬냐너가더잘써 ㅎㅎ #P01#: 5쪽 대충 의식의 흐름대로 쭉 써야지..이제 1쪽씀 ;; 5쪽 에는 네줄만 적어야지 #P02#: 안대... 뭔가분량중요할거같아 거의꽉채워서쓰셈 #P01#: 못써 쓸말업써 #P02#: 이거중간대체여?? #P01#: ㄴㄴ 그냥 과제임 그래서 더 짜증남

### 요약 >>> 행삶 과제가 너무 어려워서 5쪽을 대충 의식의 흐름대로 써야겠다고 하자 분량이 중요할 것 같으니 꽉 채워서 쓰라고 한다.

행삶 과제가 너무 어려워서 5쪽을 대충 의식의 흐름대로 써야겠다고 하자 분량이 중요할 것 같으니 꽉 채워서 쓰라고 한다.


In [3]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lwef/llama3-8B-ko-dialogue-summary-finetuned", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


adapter_config.json:   0%|          | 0.00/764 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

beomi/Llama-3-Open-Ko-8B does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


adapter_model.safetensors:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
#dialogue = '''#P01#: 나 이거 코드 오류나 #P02#: 오류에 np 써잇죠? #P01#: 어 마자 #P02#: np가 정의되지않았다 그럼 np를 정의해줘야겟죠 #P01#: import numpy as np 이렇게 하면 되나? #P02#: 네 마자요 그리고 밑에 오류 코드 봐봐요 넘파이 랜덤 모델에는 rabint가 없다 그럼 ranbint를 찾아보면돼요 #P01#: 오케이!!!! 감사합니다'''
dialogue = '''#P01#: 개별연구 주제가 뭐야? #P02#: LLM fine-tuning하는거야. #P01#: 그럼 언제 끝나? #P02#: 일요일쯤? '''
#dialogue = '''#P01#: 1학기 성적 어케 나옴 #P02#: ㅋ.ㅋ 나한테물어뭐함 Bbbbbbb 올비에 하나 비플 ㅋㅋㅋ #P01#: ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 야 그래도 #P02#: 중간다잘쳣는데 #P01#: 씨는 없네 에프도 없고 그게 어디고 #P02#: 기말에 준비물안들고가서다망햇 계산기필요하고 소수점 을직접나누기하다가 내멘탈은펑'''
formatted_prompt = prompt_template.format(dialogue=dialogue)

# 토크나이징
inputs = tokenizer(
    formatted_prompt,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 128,
    eos_token_id=tokenizer.eos_token_id, # EOS 토큰을 사용하여 명시적으로 출력의 끝을 지정.
    use_cache = True
)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
result = decoded_outputs[0]

print(result)
result = result.split('### 요약 >>>')[-1].strip()
# print(f"출력결과:{result}")


아래 대화를 요약해 주세요. 대화 형식은 '#대화 참여자#: 대화 내용'입니다.
### 대화 >>>#P01#: 개별연구 주제가 뭐야? #P02#: LLM fine-tuning하는거야. #P01#: 그럼 언제 끝나? #P02#: 일요일쯤? 

### 요약 >>>개별 연구 주제가 LLM을 미세 조정하는 것인데 일요일쯤 끝난다.



(수정 전)ROUGE 1/2/L, BertScore 계산

In [11]:
from datasets import load_dataset
from rouge_score import rouge_scorer
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import bert_score
import numpy as np
# 필요한 NLTK 데이터를 다운로드합니다.
nltk.download('punkt')

dataset = load_dataset("lsef/aihub-ko-dialogue-2", split="test")
dataset = dataset.select(range(100))
# ROUGE 스코어 계산 함수
def compute_rouge_scores(hypotheses, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    for hyp, ref in zip(hypotheses, references):
        score = scorer.score(ref, hyp)
        scores['rouge1'].append(score['rouge1'].fmeasure)
        scores['rouge2'].append(score['rouge2'].fmeasure)
        scores['rougeL'].append(score['rougeL'].fmeasure)
    return {key: np.mean(val) for key, val in scores.items()}

# BLEU 스코어 계산 함수
def compute_bleu_scores(hypotheses, references):
    smoothie = SmoothingFunction().method4
    scores = [sentence_bleu([ref], hyp, smoothing_function=smoothie) for hyp, ref in zip(hypotheses, references)]
    return np.mean(scores)

# BERT 스코어 계산 함수
def compute_bert_scores(hypotheses, references):
    P, R, F1 = bert_score.score(hypotheses, references, lang="ko", rescale_with_baseline=True)
    return {'precision': P.mean().item(), 'recall': R.mean().item(), 'f1': F1.mean().item()}

# 추론 및 평가
hypotheses = []
references = []

for example in dataset:
    dialogue = example['dialogue']
    summary = example['summary']

    formatted_prompt = f"아래 대화를 요약해 주세요. 대화 형식은 '#대화 참여자#: 대화 내용'입니다.\n### 대화 >>>{dialogue}\n\n### 요약 >>>"

    # 토크나이징
    inputs = tokenizer(
        formatted_prompt,
        return_tensors="pt",
        truncation=True,
        padding=True
    ).to("cuda")

    # 추론
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True
    )

    # 디코딩
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 추출된 요약 부분만 사용
    generated_summary = decoded_output.split('### 요약 >>>')[-1].strip()
    print(generated_summary)

    hypotheses.append(generated_summary)
    references.append(summary)

# 스코어 계산
rouge_scores = compute_rouge_scores(hypotheses, references)
bleu_score = compute_bleu_scores(hypotheses, references)
bert_scores = compute_bert_scores(hypotheses, references)

print(f"ROUGE-1: {rouge_scores['rouge1']}")
print(f"ROUGE-2: {rouge_scores['rouge2']}")
print(f"ROUGE-L: {rouge_scores['rougeL']}")
print(f"BLEU: {bleu_score}")
print(f"BERT Precision: {bert_scores['precision']}")
print(f"BERT Recall: {bert_scores['recall']}")
print(f"BERT F1: {bert_scores['f1']}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


강사님이 콘솔창에 적는 이유를 설명해 주고 있다.
한국사 시험을 10월에 보는 언니에게 응원한다.
최신 제어 시스템 책에서 맨 뒤에 답 같은 게 없냐고 하니 없다고 한다.
우리나라가 제조업 국가인 것을 알고 놀랐고, 이알피(ERP) 다음 시험은 7월이라고 한다.
전에 투다리 갔을 때 강화도에 대해 말했는데 오늘 시험이었다.
한국사 시험은 사료가 많이 나와서 어렵다.
교수님이 오셨는데 출석을 부르지 않아서 지각을 예약했다.
4학년 2학기에는 이것저것 해야 되면 더 정신없고 막막할 것 같다고 하자 이번 학기에 한다고 한다.
12월 22일 토요일에 경찰 시험을 보러 가는데 접수 가능하냐고 묻고 있다.
용의자의 아들이 92년생인데 살인의 추억을 엄청 유명하게 보고 자란 세대인데 아빠가 그 범인이라 부끄러워서 어디 가서 아빠 안 계시다고 하고 살 것 같다.
수업 시간에 발표하는 사람만 대답하라고 해서 탈탈 털렸다.
수업 신청을 하려고 결제를 했는데 결제가 안돼서 무통장 입금을 했는데 0원이니까 안 넣어도 될 것 같다.
연예인들이 건물을 많이 사고 팔고 하는 재테크를 하는데 부러워한다.
전공선택(전선) 수업을 다 빼고 21학점을 채울 것인지에 대해 이야기하고 있다.
수업 시간에 오레오를 배웠는데 만드는 과정이 재미있다.
중고나라 범인 잡힌 것에 대해 이야기하고 있다.
주간을 다니면 학교가느라 어쩔 수 없이 나와야 하니까 끝나고 독서실로 갈 명분이 있는데 야간을 다니면 안 그럴 것 같아서 게으른 것 같다고 한다.
대학원 수업 때문에 다른 학교의 교과과정을 알아봤는데 우리 학교는 너무 다방면의 수업이 많아서 작곡과 같지 않은 느낌이 났다.
롯데월드에서 코로나 확진자가 나와서 롯데월드가 망했다고 한다.
대구에서 수성구만 새누리당 김문수가 떨어졌고 인천에서도 민주당이 많이 당선된 것 같다.
교수님이 병원에 갈 거면 엄청 공부해야 한다고 하셨는데 실습 때 관리 순서가 달마다 바뀌어서 외우기 싫다.
지진이 있었는데 느끼지도 못했다고 한다.
통영에 신청하고 싶은 활동이 있는데 2학기

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

ROUGE-1: 0.11566666666666667
ROUGE-2: 0.02
ROUGE-L: 0.11566666666666667
BLEU: 0.2735379415420359
BERT Precision: 0.7953600883483887
BERT Recall: 0.7908473014831543
BERT F1: 0.7922749519348145


(수정 후) 한국어에 맞게 전처리 후 ROUGE 1/2/L (korouge 사용)
https://github.com/HeegyuKim/korouge

In [11]:
!pip install korouge_score konlpy bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [12]:
from datasets import load_dataset
from korouge_score import rouge_scorer
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from konlpy.tag import Okt
import bert_score
import numpy as np
import re

nltk.download('punkt')

dataset = load_dataset("lwef/aihub-ko-dialogue-2", split="test")
dataset = dataset.select(range(100))

# 요약문 데이터 전처리
def remove(text):
    # {단어}({줄임말단어}){조사} 형태를 {단어}{조사} 형태로
    t = re.sub(r'\([^)]*\)', '', text)
    # 특수문자 삭제
    t = re.sub(r'[^\w\s]', '', t)
    return t

def preprocess_korean(text, okt):
    # 형태소 분석
    morphs = okt.morphs(text, stem= True)
    return ' '.join(morphs)

# ROUGE score 계산 함수
def compute_rouge_scores(hypotheses, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    okt = Okt()
    for hyp, ref in zip(hypotheses, references):
        hyp = remove(hyp)
        ref = remove(ref)
        hyp = preprocess_korean(hyp, okt)
        ref = preprocess_korean(ref, okt)
        score = scorer.score(ref, hyp)
        scores['rouge1'].append(score['rouge1'].fmeasure)
        scores['rouge2'].append(score['rouge2'].fmeasure)
        scores['rougeL'].append(score['rougeL'].fmeasure)
    return {key: np.mean(val) for key, val in scores.items()}

# BERT score 계산 함수
def compute_bert_scores(hypotheses, references):
    P, R, F1 = bert_score.score(hypotheses, references, lang="ko", rescale_with_baseline=True)
    return {'precision': P.mean().item(), 'recall': R.mean().item(), 'f1': F1.mean().item()}

# 추론 및 평가
hypotheses = []
references = []

for example in dataset:
    dialogue = example['dialogue']
    summary = example['summary']

    formatted_prompt = f"아래 대화를 요약해 주세요. 대화 형식은 '#대화 참여자#: 대화 내용'입니다.\n### 대화 >>>{dialogue}\n\n### 요약 >>>"

    # 토크나이징
    inputs = tokenizer(
        formatted_prompt,
        return_tensors="pt",
        truncation=True,
        padding=True
    ).to("cuda")

    # 추론
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True
    )

    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    generated_summary = decoded_output.split('### 요약 >>>')[-1].strip()
    print(generated_summary)

    hypotheses.append(generated_summary)
    references.append(summary)

# 스코어 계산
bert_scores = compute_bert_scores(hypotheses, references)
rouge_scores = compute_rouge_scores(hypotheses, references)

print("### BERT Score")
print(f"BERT Precision: {bert_scores['precision']}")
print(f"BERT Recall: {bert_scores['recall']}")
print(f"BERT F1: {bert_scores['f1']}")

print("\n###ROUGE Score")
print(f"ROUGE-1: {rouge_scores['rouge1']}")
print(f"ROUGE-2: {rouge_scores['rouge2']}")
print(f"ROUGE-L: {rouge_scores['rougeL']}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Generating train split:   0%|          | 0/13119 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1312 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/328 [00:00<?, ? examples/s]

강사님이 콘솔창에 적는 이유를 설명해 주고 있다.
한국사 시험을 10월에 보는 언니에게 응원한다.
최신 제어 시스템 책에서 맨 뒤에 답 같은 게 없냐고 하니 없다고 한다.
우리나라가 제조업 국가인 것을 알고 놀랐고, 이알피(ERP) 다음 시험은 7월이라고 한다.
전에 투다리 갔을 때 강화도에 대해 말했는데 오늘 시험이었다.
한국사 시험은 사료가 많이 나와서 어렵다.
교수님이 오셨는데 출석을 부르지 않아서 지각을 예약했다.
4학년 2학기에는 이것저것 해야 되면 더 정신없고 막막할 것 같다고 하자 이번 학기에 한다고 한다.
12월 22일 토요일에 경찰 시험을 보러 가는데 접수 가능하냐고 묻고 있다.
용의자의 아들이 92년생인데 살인의 추억을 엄청 유명하게 보고 자란 세대인데 아빠가 그 범인이라서 부끄러워서 어디 가서 아빠 안 계시다고 하고 살 것 같다.
수업 시간에 발표하는 사람만 대답하라고 해서 탈탈 털렸다.
수업 신청을 하려고 결제를 했는데 결제가 안돼서 무통장 입금을 했는데 0원이니까 안 넣어도 될 것 같다.
연예인들이 건물을 많이 사고 팔고 하는 재테크를 하는 것에 대해 이야기한다.
전공선택(전선) 1개만 들을 것 같고, 오씨유(OCU) 2개 중에 들을 만한 게 있냐고 한다.
수업 시간에 오레오를 배웠는데 만드는 과정이 재미있다.
중고나라 범인 잡힌 것에 대해 이야기하고 있다.
주간을 다니면 학교가느라 어쩔 수 없이 나와야 하니까 끝나고 독서실로 갈 명분이 있는데 야간을 다니면 안 그럴 것 같아서 게으른 것 같다고 한다.
대학원 수업 때문에 다른 학교의 교과과정을 알아봤는데 우리 학교는 너무 다방면의 수업이 많아서 작곡과 같지 않은 느낌이 났다.
롯데월드에서 코로나 확진자가 나와서 롯데월드가 망했다고 한다.
대구에서 수성구만 새누리당 김문수가 떨어졌고 인천에서도 민주당이 많이 당선된 것 같다.
교수님이 병원에 갈 거면 엄청 공부를 해야 한다고 하셨는데 실습 때 관리 순서가 달마다 바뀌어서 외우기 싫다.
지진이 있었는데 느끼지 못했다고 한다.
통영에 신청하고

...

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

### BERT Score
BERT Precision: 0.7952985167503357
BERT Recall: 0.7901075482368469
BERT F1: 0.7918769717216492

###ROUGE Score
ROUGE-1: 0.4167147409430521
ROUGE-2: 0.22379921682079196
ROUGE-L: 0.3785368919466586


아래는 huggingface 모듈 load_metric("rouge")를 사용한 ROUGE score 계산. 이것도 영어 대상이라 결과가 나오지 않는다.

In [2]:
from datasets import load_metric

predictions = ["한국사 시험은 사료가 많이 나와서 어렵다."]
references = ["한국사에 사료가 많이 나와서 어렵다고 한다."]
rouge = load_metric("rouge")
results = rouge.compute(predictions=predictions, references=references)

print(results)

<ipython-input-2-91e787afc79f>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] ㅛ
The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
{'rouge1': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)), 'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)), 'rougeL': AggregateScore(lo

### GPT 3.5를 이용한 요약 평가

openAI API 테스트

참고
https://github.com/openai/openai-python/issues/987
https://github.com/openai/openai-python/blob/main/examples/demo.py

In [13]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 11.0 MB/s eta 0:00:00


In [16]:
print(hypotheses[2])
#print(hypotheses[1])
print(references[2])

최신 제어 시스템 책에서 맨 뒤에 답 같은 게 없냐고 하니 없다고 한다.
최신제어 시스템 책에서 답지가  없는 거 같다고 하였다.


### 다른 모델 불러오기

In [15]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "SkyOrbis/SKY-Ko-Llama3-8B-lora", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

SkyOrbis/SKY-Ko-Llama3-8B-lora does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


추론 테스트 코드

In [6]:
#dialogue = '''#P01#: 나 이거 코드 오류나 #P02#: 오류에 np 써잇죠? #P01#: 어 마자 #P02#: np가 정의되지않았다 그럼 np를 정의해줘야겟죠 #P01#: import numpy as np 이렇게 하면 되나? #P02#: 네 마자요 그리고 밑에 오류 코드 봐봐요 넘파이 랜덤 모델에는 rabint가 없다 그럼 ranbint를 찾아보면돼요 #P01#: 오케이!!!! 감사합니다'''
#dialogue = '''#P01#: 개별연구 주제가 뭐야? #P02#: LLM fine-tuning하는거야. #P01#: 그럼 언제 끝나? #P02#: 일요일쯤? '''
#dialogue = '''#P01#: 1학기 성적 어케 나옴 #P02#: ㅋ.ㅋ 나한테물어뭐함 Bbbbbbb 올비에 하나 비플 ㅋㅋㅋ #P01#: ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 야 그래도 #P02#: 중간다잘쳣는데 #P01#: 씨는 없네 에프도 없고 그게 어디고 #P02#: 기말에 준비물안들고가서다망햇 계산기필요하고 소수점 을직접나누기하다가 내멘탈은펑'''
#dialogue = '''#P01#: 아 행삶 과제 너무 어려워... 5쪽 쓸게 없는데 ㅡㅡ #P02#: 몬냐몬냐너가더잘써 ㅎㅎ #P01#: 5쪽 대충 의식의 흐름대로 쭉 써야지..이제 1쪽씀 ;; 5쪽 에는 네줄만 적어야지 #P02#: 안대... 뭔가분량중요할거같아 거의꽉채워서쓰셈 #P01#: 못써 쓸말업써 #P02#: 이거중간대체여?? #P01#: ㄴㄴ 그냥 과제임 그래서 더 짜증남'''
formatted_prompt = prompt_template.format(dialogue=dialogue)

# 토크나이징
inputs = tokenizer(
    formatted_prompt,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 128,
    eos_token_id=tokenizer.eos_token_id, # EOS 토큰을 사용하여 명시적으로 출력의 끝을 지정.
    use_cache = True
)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
result = decoded_outputs[0]

print(result)
result = result.split('### 요약 >>>')[-1].strip()
print(result)
# print(f"출력결과:{result}")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



아래 대화를 요약해 주세요. 대화 형식은 '#대화 참여자#: 대화 내용'입니다.
### 대화 >>>#P01#: 나 이거 코드 오류나 #P02#: 오류에 np 써잇죠? #P01#: 어 마자 #P02#: np가 정의되지않았다 그럼 np를 정의해줘야겟죠 #P01#: import numpy as np 이렇게 하면 되나? #P02#: 네 마자요 그리고 밑에 오류 코드 봐봐요 넘파이 랜덤 모델에는 rabint가 없다 그럼 ranbint를 찾아보면돼요 #P01#: 오케이!!!! 감사합니다

### 요약 >>>#P01#: 오류를 해결하기 위해 numpy를 가져옵니다. #P02#: numpy를 import하고 ranbint 함수를 찾아서 문제를 해결합니다.



50개의 테스트 데이터에 대해 요약문 생성, another_model_output에 저장

In [17]:
from datasets import load_dataset
dataset = load_dataset("lsef/aihub-ko-dialogue-2", split="test")
dataset = dataset.select(range(50))

another_model_output = []

for example in dataset:
    dialogue = example['dialogue']
    summary = example['summary']

    formatted_prompt = f"아래 대화를 요약해 주세요. 대화 형식은 '#대화 참여자#: 대화 내용'입니다.\n### 대화 >>>{dialogue}\n\n### 요약 >>>"

    # 토크나이징
    inputs = tokenizer(
        formatted_prompt,
        return_tensors="pt",
        truncation=True,
        padding=True
    ).to("cuda")

    # 추론
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True
    )

    # 디코딩
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 추출된 요약 부분만 사용
    generated_summary = decoded_output.split('### 요약 >>>')[-1].strip()
    print(generated_summary)

    another_model_output.append(generated_summary)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#대화 참여자#: R에서 콘솔창에 입력하면 콘솔창에 결과가 나온다는 것을 배웠습니다. 또한, R에서는 콘솔창에 입력하면 콘솔창에 결과가 나올 뿐만 아니라 콘솔창에만 입력할 수 있다는 것도 배웠습니다. 또한, R에서는 콘솔창에 입력하면 콘솔창에 결과가 나올 뿐만 아니라 콘솔창에만 입력할 수 있다는 것도 배웠습니다. 콘솔창에 입력하면 콘솔창에 결과가 나온다는 것을 배웠습니다.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#대화 참여자#: 한국사 공부를 해야하는 이유와 언니가 10월에 시험을 볼 예정이라는 정보를 공유합니다.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

#대화 참여자#: 대화 내용
#대화 참여자#: 미국이나 유럽은 교육이 잘 되어 있어서 외국인들이 자국민보다 더 유리합니다.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


속초 고성의 강을 흐르는 강에 불이 붙은 상황에 대해 이야기하고 있습니다. 불이 심하게 불어오고 사람들은 대피하는 모습이 너무 아프다. 소방관들이 대응하고 있다.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#대화 참여자#: 대화 내용
#P01#: 낙타 근처 가지말고 막 짜낸 낙타유 마시지말고 응?
#P02#: ㅋㄱㄱㅋㄱㄱㄱㄱㄱㄱ 본적도없다ㅋㅋ
#P01#: ㅋㅋㅋㅋㅋㄱㄱ정부에서저렇게말했어 예방법이라고 미친듯 ㅋㅋㅋㅋㅋㅋ
#P02#: 보고빡쳤다ㅋㅋㄱㅋ 이상한거같어 후...
#P01#: ㅋㅋㅋㅋㅋㅋㅋ 사람들댓글드립
#P01#: 코딩 책을 읽고 컴퓨터가 없어서 문제가 생겼습니다. #P02#: 책을 샀어요? #P01#: 네, 샀어요. #P02#: 프로그램이 없어요?


gpt 평가 코드: dialogue fine tuned model만 평가

In [25]:
from openai import OpenAI
from datasets import load_dataset
import json
import ast
gpt_template = '''다음 대화 내용을 기반으로 요약문의 품질을 객관적으로 평가해 주세요.
1. 요약의 정확성 2. 요약의 일관성 3. 요약의 문법적 정확성
위의 세 가지 기준에서 각각 1에서 10 사이의 점수로 평가해 주세요. 1은 최악, 10은 최고입니다. 대답의 형식은 반드시 {{'accuracy': {{점수}}, 'coherence': {{점수}}, 'grammar': {{점수}}}} 여야 합니다.
대화: {dialogue}
요약: {summary}'''
OPENAI_API_KEY = ""

# openai API 키 인증
client = OpenAI(api_key=OPENAI_API_KEY)

model = "gpt-3.5-turbo"

dataset = load_dataset("lsef/aihub-ko-dialogue-2", split="test")
test_dataset = dataset.select(range(50))

def evaluate_summary(dialogue, summary):
    prompt = gpt_template.format(dialogue=dialogue, summary=summary)

    messages = [{
        "role": "system",
        "content": "You are a helpful assistant."
    }, {
        "role": "user",
        "content": prompt
    }]

    response = client.chat.completions.create(
          model=model,
          messages=messages
      )

    answer = response.choices[0].message.content.strip()
    print(answer)
    return json.loads(answer.replace("'", '"'))

# 평가 결과를 저장할 리스트
evaluation_results = []

for i, example in enumerate(test_dataset):
    dialogue = example['dialogue']
    summary = hypotheses[i]
    if i < 2:  # 첫 두 개만 출력
        print(f"Dialogue: {dialogue}\nSummary: {summary}")
    result = evaluate_summary(dialogue, summary)
    evaluation_results.append(result)

# 평가 결과의 평균 계산
def calculate_average(results):
    total_accuracy = 0
    total_coherence = 0
    total_grammar = 0
    count = len(results)

    for result in results:
        total_accuracy += result['accuracy']
        total_coherence += result['coherence']
        total_grammar += result['grammar']

    return {
        'accuracy': total_accuracy / count,
        'coherence': total_coherence / count,
        'grammar': total_grammar / count
    }

average_scores = calculate_average(evaluation_results)
print("Average Scores:", average_scores)


Dialogue: #P01#: 지금 하는게 그냥 R을 사용하는 법을 배우는거고 그걸 어떻게 쓰는지 직접 타이핑해보고 끝인거지?
#P02#: 그쵸 나중에 전처리 할때 저런것들은 쓸일 있으니까 알아두는게 좋을듯
#P01#: 아하 오키오키 근데 강사님은 왜 콘솔창에 적어?
#P02#: 암데나 해도 상관없어요
#P01#: 아하 근데 원래 콘솔창은 내가 위에있는 창에 입력하면 거기에 결과보여주는데 맞지
#P02#: 막 실행오류 뜨거나 뭐 그런거 뜨던곳 자바 아래에 콘솔창이랑 똑같아요 근데 r은 콘솔창에도 타이핑을 할 수 있다는게 차이지
#P01#: 아 오케오케
Summary: 강사님이 콘솔창에 적는 이유를 설명해 주고 있다.
{'accuracy': 2, 'coherence': 7, 'grammar': 6}
Dialogue: #P01#: ㅋㅋㅋㅋㅋㅋ 행복하다 하나라서
#P02#: 한국사해야지요..^__^
#P01#: 악!!!
#P02#: 해보세여...!!! 한국사!!! 화이팅!!!
#P01#: ㅋㅋㅋㅋㅋㅋ 언니도 10월에 시험 보는거지요?
#P02#: 네! 저도 신청했습니다!!
Summary: 한국사 시험을 10월에 보는 언니에게 응원한다.
{'accuracy': 3, 'coherence': 6, 'grammar': 6}
{'accuracy': 5, 'coherence': 4, 'grammar': 2}
{'accuracy': 7, 'coherence': 7, 'grammar': 8}
{'accuracy': 3, 'coherence': 4, 'grammar': 5}
{'accuracy': 6, 'coherence': 5, 'grammar': 7}
{'accuracy': 2, 'coherence': 3, 'grammar': 2}
{'accuracy': 7, 'coherence': 8, 'grammar': 7}
{'accuracy': 7, 'coherence': 8, 'grammar': 8}
{'accuracy': 5, 'coherence': 7, 'grammar': 6}
{

GPT 평가 코드: 평가 데이터의 요약문, dialogue fine-tuned 요약문, 유사한 다른 모델의 생성 요약문 3가지 비교

In [18]:
print(hypotheses[0])
print(references[0])
print(another_model_output[0])

강사님이 콘솔창에 적는 이유를 설명해 주고 있다.
알(R)을 사용하는 법을 배우는 데 알(R)은 콘솔 창에도 타이핑을 할 수 있다는 차이점이 있다.
#대화 참여자#: R에서 콘솔창에 입력하면 콘솔창에 결과가 나온다는 것을 배웠습니다. 또한, R에서는 콘솔창에 입력하면 콘솔창에 결과가 나올 뿐만 아니라 콘솔창에만 입력할 수 있다는 것도 배웠습니다. 또한, R에서는 콘솔창에 입력하면 콘솔창에 결과가 나올 뿐만 아니라 콘솔창에만 입력할 수 있다는 것도 배웠습니다. 콘솔창에 입력하면 콘솔창에 결과가 나온다는 것을 배웠습니다.
